# Calculate GWP and make tabels for the paper...

Description of the experiments. bla bla

To do:
- Add references  https://jupyterbook.org/en/stable/content/citations.html
- Write a detailed explanation of the method inkluding references
- Loop, so drydep 80, 50 and no adjustments are run at the same tima

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.float_format', lambda x: '{:,.4g}'.format(x) if abs(x)<1000 else '{:,.0f}'.format(x))

Input and output path:

In [2]:
path = r"./input/"
outputpath= r"./output/"

Constants:

In [3]:
#AGWP100_CO2 [mW yr m-2 Tg-1] Source: REF.
agwp100_CO2 = 0.09170

#CH4 tau_strat[yr] Source: REF
tau_strat = 120.0

#CH4 tau_soil [yr] Source: REF
tau_soil = 160.0

#Specific RF for CH4 [mW m-2 ppb-1] Etminan et al., 2016
spec_rf_ch4 = 0.44300

As there is large uncertanty in drydeposition values, we will, in addition to use the drydeposition calculated by the model (but not used in the model, as the surface concentration is fixed), set the drydeposition in the control run to a high and low value based on literature (IS 80 and 50 OK?). This will result in a range of GWP values. Adjust the perturbations with the same relative factors as in the control.

In [4]:
adjust_drydep = False
if(adjust_drydep):
    drydep = 50.0
    outputpath = outputpath + 'drydep_'+ f'{drydep:.0f}_'

## Read input data

To run this notebook, preprosessed output from the 3 set of simulations and put into a folder named input.
- CTRL: Control simulation, fixed surface concentration
- 10H2: Hydrogen perturbation, surface H_2 increased by 10%. Run to steady state.
- 10CH4: Methane perturbation, surface CH_4 increased by 10%. Run to steady state.

In addition, emission driven runs are added. GFDL-emi: description of this experiment.

Add table with length of simulation and over how many years the model results are averaged. Data in input is based on monthly data for this averaged data.

## Hydrogen results

### H2 burden [Tg]:

In [5]:
file = 'H2_burden.txt'
df_h2_burden = pd.read_csv(path + file, sep=';',index_col=0,header=0)
delta = df_h2_burden.loc['10H2']-df_h2_burden.loc['CTRL']
delta.name = 'deltaH2'
df_h2_burden = df_h2_burden.append(delta)
df_h2_burden

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,GFDL
Scenario,,,,,
CTRL,196.2,194.4,194.4,191.7,187.2
10H2,214.9,213,213,655.6,205.4
10CH4,196.8,194.9,194.9,nan,187.6
deltaH2,18.65,18.6,18.6,463.9,18.17


### H2 loss

### H2 drydep [Tg/yr]

The models diagnose drydeposition based on their own schemes, but this is not used in the concentration driven runs.

In [6]:
file = 'H2_drydep.txt'
df_h2_drydep = pd.read_csv(path + file, sep=';',index_col=0,header=0)
df_h2_drydep

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,GFDL
Scenario,,,,,
CTRL,59.52,72.46,52.06,54.66,56.77
10H2,65.47,79.7,57.23,203.7,62.43
10CH4,59.52,72.47,52.07,nan,56.77


In [7]:
if(adjust_drydep):
    print('NB drydep adjusted')
    
    #Adjust by the relative adjustment in the control simulations
    adjust = drydep/df_h2_drydep.loc['CTRL']
    df_h2_drydep = df_h2_drydep*adjust
    
    print(df_h2_drydep)

### H2 atm. loss [Tg/yr]

In [8]:
file = 'H2_atm_loss.txt'
df_h2_atmloss = pd.read_csv(path + file, sep=';',index_col=0,header=0)
df_h2_atmloss


,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,GFDL
Scenario,,,,,
CTRL,28.39,29.51,22.13,22.12,21.53
10H2,30.97,32.19,24.15,69.45,23.54
10CH4,27.64,28.6,21.63,nan,21


### H2 total loss [Tg/yr]:

In [9]:
df_h2_loss = df_h2_atmloss + df_h2_drydep
df_h2_loss


,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,GFDL
Scenario,,,,,
CTRL,87.91,102,74.19,76.78,78.3
10H2,96.44,111.9,81.38,273.1,85.96
10CH4,87.15,101.1,73.7,nan,77.78


### H2 production

### H2 atm. prod [Tg/yr]

In [10]:
file = 'H2_atm_prod.txt'
df_h2_atmprod = pd.read_csv(path + file, sep=';',index_col=0,header=0)
df_h2_atmprod

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,GFDL
Scenario,,,,,
CTRL,56.32,34.63,46.68,44.68,44.66
10H2,56.23,34.52,46.61,43.12,44.6
10CH4,59.16,37.1,48.91,nan,46.7


### Estimated emissions (Total loss = Total prod + emission)

In surface concentration driven runs, there are two unknow, emissions and drydeposition. 
Emission driven runs, are driven by emission estimates and use drydeposition scheme to calculate the concentration at the surface. Althogh the emission driven runs use drydep and emissions, we adjust the drydep as well to take into account uncertainties in emissions and drydeposition on the GWP values.

In [11]:
df_h2_estemis = df_h2_atmloss.loc['CTRL'] + df_h2_drydep.loc['CTRL'] - df_h2_atmprod.loc['CTRL']
df_h2_estemis

OSLOCTM3      31.58
WACCM6-2deg   67.34
INCA          27.51
GFDL-emi       32.1
GFDL          33.64
Name: CTRL, dtype: float64

### H2 surface concentration [ppb]

In [12]:
file = 'H2_surfconc.txt'
df_h2_surfconc = pd.read_csv(path + file, sep=';',index_col=0,header=0)
delta = df_h2_surfconc.loc['10H2']-df_h2_surfconc.loc['CTRL']
delta.name = 'deltaH2'
df_h2_surfconc = df_h2_surfconc.append(delta)
df_h2_surfconc

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,GFDL
Scenario,,,,,
CTRL,531.7,531.8,531.8,533.6,525.2
10H2,584.9,585,585,"1,880",577.6
deltaH2,53.2,53.18,53.2,"1,346",52.46


### H2 lifetime [yr]

In [13]:
df_h2_lifetime = df_h2_burden.drop('deltaH2')/df_h2_loss
df_h2_lifetime

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,GFDL
Scenario,,,,,
CTRL,2.232,1.906,2.62,2.497,2.391
10H2,2.228,1.904,2.617,2.401,2.389
10CH4,2.258,1.929,2.645,nan,2.412


### H2 atmospheric lifetime [yr]

In [14]:
df_h2_atm_lifetime = df_h2_burden.drop('deltaH2')/df_h2_atmloss
df_h2_atm_lifetime

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,GFDL
Scenario,,,,,
CTRL,6.911,6.587,8.786,8.665,8.693
10H2,6.937,6.616,8.82,9.441,8.726
10CH4,7.121,6.816,9.013,nan,8.932


### H2 flux (burden dived by lifetime) [Tg/yr]

Add a description of the delta flux here. Iclude feedback

In [15]:
df_h2_flux = df_h2_burden.drop('deltaH2')/df_h2_lifetime
#Add delta flux 10H2:
delta = df_h2_flux.loc['10H2']-df_h2_flux.loc['CTRL']
delta.name = 'deltaH2'
df_h2_flux = df_h2_flux.append(delta)
#Add delta flux 10CH4:
delta = df_h2_flux.loc['10CH4']-df_h2_flux.loc['CTRL']
delta.name = 'deltaCH4'
df_h2_flux = df_h2_flux.append(delta)
df_h2_flux


,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,GFDL
Scenario,,,,,
CTRL,87.91,102,74.19,76.78,78.3
10H2,96.44,111.9,81.38,273.1,85.96
10CH4,87.15,101.1,73.7,nan,77.78
deltaH2,8.536,9.921,7.195,196.3,7.661
deltaCH4,-0.754,-0.8981,-0.4905,nan,-0.5231


Description of the negative deltaCH4. In the methane run, the hydrogen surface concentration is kept fixed. Enhancing methane would influence H2. The hydrogen concentration would have increased, but since we run with fixed concentration, there is a negative flux to compensate. So the increased flux in H2 due to methane is -1*deltaCH4.

## Methane results

### CH4 burden [Tg]

In [16]:
file = 'CH4_burden.txt'
df_ch4_burden = pd.read_csv(path + file, sep=';',index_col=0,header=0)
delta = df_ch4_burden.loc['10CH4']-df_ch4_burden.loc['CTRL']
delta.name = 'deltaCH4'
df_ch4_burden = df_ch4_burden.append(delta)
df_ch4_burden

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,GFDL
Scenario,,,,,
CTRL,"4,995","5,006","4,994","4,935","4,857"
10H2,"4,995","5,007","4,995","4,935","4,857"
10CH4,"5,495","5,508","5,494",nan,"5,343"
deltaCH4,500.1,501.8,499.5,nan,486


### CH4 atmospheric loss [Tg/yr]

In [17]:
file = 'CH4_loss.txt'
df_ch4_loss = pd.read_csv(path + file, sep=';',index_col=0,header=0)
df_ch4_loss

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,GFDL
Scenario,,,,,
CTRL,683,742.3,549.6,582.3,570
10H2,680.3,739,547.5,537.1,568.1
10CH4,728,788.1,587.6,nan,610.2


### CH4 surface concentration [ppb]

In [18]:
file = 'CH4_surfconc.txt'
df_ch4_surfconc = pd.read_csv(path + file, sep=';',index_col=0,header=0)
delta = df_ch4_surfconc.loc['10CH4']-df_ch4_surfconc.loc['CTRL']
delta.name = 'deltaCH4'
df_ch4_surfconc = df_ch4_surfconc.append(delta)
df_ch4_surfconc

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,GFDL
Scenario,,,,,
CTRL,"1,813","1,808","1,813","1,804","1,789"
10CH4,"1,994","1,989","1,994",nan,"1,967"
deltaCH4,181.3,180.8,181.3,nan,178.9


### CH4 lifetime [yr]

In [19]:
df_ch4_lifetime = df_ch4_burden.drop('deltaCH4')/df_ch4_loss
df_ch4_lifetime

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,GFDL
Scenario,,,,,
CTRL,7.314,6.744,9.088,8.476,8.52
10H2,7.342,6.775,9.123,9.189,8.548
10CH4,7.549,6.99,9.35,nan,8.755


### Total CH4 lifetime [yr]

In [20]:
df_ch4_tot_lifetime = 1.0/(1.0/df_ch4_lifetime + 1.0/tau_strat + 1.0/tau_soil)
df_ch4_tot_lifetime

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,GFDL
Scenario,,,,,
CTRL,6.609,6.14,8.024,7.543,7.579
10H2,6.632,6.166,8.052,8.103,7.601
10CH4,6.8,6.343,8.228,nan,7.764


### CH4 flux (burden dived by total lifetime) [Tg/yr]

In [21]:
df_ch4_flux = df_ch4_burden.drop('deltaCH4')/df_ch4_tot_lifetime
#Add delta CH4 flux 10H2
delta = df_ch4_flux.loc['10H2']-df_ch4_flux.loc['CTRL']
delta.name = 'deltaH2'
df_ch4_flux = df_ch4_flux.append(delta)
#Add delta CH4 flux 10CH4
delta = df_ch4_flux.loc['10CH4']-df_ch4_flux.loc['CTRL']
delta.name = 'deltaCH4'
df_ch4_flux = df_ch4_flux.append(delta)
df_ch4_flux


,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,GFDL
Scenario,,,,,
CTRL,755.8,815.3,622.4,654.2,640.8
10H2,753.1,812,620.3,609,639
10CH4,808.1,868.4,667.7,nan,688.1
deltaH2,-2.671,-3.378,-2.105,-45.18,-1.888
deltaCH4,52.29,53.05,45.33,nan,47.3


## Ozone results

### Ozone delta DU troposphere

The tropopause definition is the model layer in the control simulation where 150 ppbv ozone are reached. For GFDL-emi 10H2 is H2 perturbation and 10CH4 is H2+CH4 pert minus the H2 pert.

In [22]:
file = 'ozone_du_trop.txt'
df_ozone_du_trop = pd.read_csv(path+file, sep=';',index_col=0,header=0)
df_ozone_du_trop

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,GFDL
Scenario,,,,,
10H2,0.03846,0.02758,0.02928,0.7292,0.05754
10CH4,0.8309,0.8596,0.5394,0.7624,0.6618


### Ozone delta DU stratosphere

In [23]:
file = 'ozone_du_strat.txt'
df_ozone_du_strat = pd.read_csv(path+file, sep=';',index_col=0,header=0)
df_ozone_du_strat

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,GFDL
Scenario,,,,,
10H2,0.009863,-0.01275,-0.02743,-0.2264,0.06669
10CH4,0.8294,0.7731,0.3524,1.073,0.7768


### Stratospheric ozone 60S-60N

In [24]:
file = 'ozone_du_strat_6060.txt'
df_ozone_du_strat_6060 = pd.read_csv(path+file, sep=';',index_col=0,header=0)
df_ozone_du_strat_6060

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,GFDL
Scenario,,,,,
10H2,0.007298,-0.01097,-0.02112,-0.2184,0.02436
10CH4,0.6299,0.6101,0.2767,0.6759,0.5702


### Stratospheric ozone 90S-60S and 60N-90N

In [25]:
file = 'ozone_du_strat_gt60.txt'
df_ozone_du_strat_gt60 = pd.read_csv(path+file, sep=';',index_col=0,header=0)
df_ozone_du_strat_gt60

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,GFDL
Scenario,,,,,
10H2,0.002565,-0.001779,-0.006304,-0.007968,0.04233
10CH4,0.2325,0.163,0.07574,0.397,0.2066


### Ozone RF

Ozone RF is calculated using a radiative kernel (Skeie et al 2020) and the modelled changes in ozone concentration. For GFDL-emi 10H2 RF is forcing calculated in the H2 perturbation and 10CH4 the forcing calculated by the H2+CH4 pert minus the H2 pert.

In [26]:
file = 'ozone_rf.txt'
df_ozone_rf = pd.read_csv(path+file, sep=';',index_col=0,header=0)
df_ozone_rf

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,GFDL
Scenario,,,,,
10H2,1.871,1.412,1.117,34.01,3.456
10CH4,40.97,41.7,22.58,37.59,33.34


### Stratospheric H2O RF [mW m-2]

Stratospheric H2O RF calculated offline. Full calculations will be performed. So far only LW. Multiply by 0.85

In [27]:
file = 'H2O_rf.txt'
df_h2o_rf = pd.read_csv(path+file, sep=';',index_col=0,header=0)

#'NBNBNB multiply these numbers by 0.85
df_h2o_rf = df_h2o_rf.multiply(0.85)
df_h2o_rf

,OSLOCTM3,WACCM6-2deg,INCA,GFDL
Scenario,,,,
10H2,1.351,0.5185,0.3995,nan
10CH4,9.18,2.805,3.4,nan


In [28]:
#NBNBBN not multiply these numbers by 0.85'
file = 'H2O_rf_gfdl.txt'
h2o_rf_gfdl = pd.read_csv(path + file, sep=',',index_col=0,header=0)
h2o_rf_gfdl

,GFDL
H2 [mW m-2],28.26
H2+CH4 [mW m-2],53.12



# Start calculations:

ADD DESCRIPTION OF GWP CALCULATION steady-state = integral of a pulse. Here we can cite :cite:p:`Prather2002a` and :cite:p:`Prather2007a`


### Change in H2 surface conc. caused by 1 Tg H2/yr [ppb yr Tg-1]

This is not used for the GWP calculation. Only for the per flux table and for the feedback factor calulations.

In [29]:
df_surf_h2_per_h2_flux = df_h2_surfconc.loc['deltaH2']/df_h2_flux.loc['deltaH2']
df_surf_h2_per_h2_flux.name = 'surf_h2_per_h2_flux'
df_surf_h2_per_h2_flux

OSLOCTM3      6.233
WACCM6-2deg    5.36
INCA          7.395
GFDL-emi      6.859
GFDL          6.847
Name: surf_h2_per_h2_flux, dtype: float64

### Change in CH4 flux caused by 1 TgH2 /yr (includes H2 feedback) [Tg CH4/Tg H2]:

The ch4_flux is multiplied by -1 (see above, or move description here)... Why this includes feedback...

In [30]:
df_ch4_flux_per_h2_flux = -1.0*df_ch4_flux.loc['deltaH2']/df_h2_flux.loc['deltaH2']
df_ch4_flux_per_h2_flux.name = 'ch4_flux_per_h2_flux'
df_ch4_flux_per_h2_flux

OSLOCTM3       0.313
WACCM6-2deg   0.3405
INCA          0.2925
GFDL-emi      0.2301
GFDL          0.2464
Name: ch4_flux_per_h2_flux, dtype: float64

### Change in CH4 surface conc. caused by 1 Tg/yr CH4 [ppb yr/Tg CH4]

In [31]:
df_surf_ch4_per_ch4_flux =  df_ch4_surfconc.loc['deltaCH4']/df_ch4_flux.loc['deltaCH4']
df_surf_ch4_per_ch4_flux.name = 'surf_ch4_per_ch4_flux'
df_surf_ch4_per_ch4_flux

OSLOCTM3      3.467
WACCM6-2deg   3.409
INCA          3.999
GFDL-emi        nan
GFDL          3.782
Name: surf_ch4_per_ch4_flux, dtype: float64

### Change in CH4 surface concentration per emission H2 [ppb yr /Tg H2]

In [32]:
df_surf_ch4_per_h2_flux = df_surf_ch4_per_ch4_flux*df_ch4_flux_per_h2_flux
df_surf_ch4_per_h2_flux.name = 'surf_ch4_per_h2_flux'

#Add GFDL:
#Increase in surface concentration CH4:
#1808 to 2005 ppbv (REF Paolot)
df_surf_ch4_per_h2_flux['GFDL-emi']= (2005.-1808.)/df_h2_flux['GFDL-emi'].loc['deltaH2']
df_surf_ch4_per_h2_flux

OSLOCTM3       1.085
WACCM6-2deg    1.161
INCA            1.17
GFDL-emi       1.004
GFDL          0.9319
Name: surf_ch4_per_h2_flux, dtype: float64

### Change in H2 flux caused by 1 TgCH4/yr [Tg H2/Tg CH4]

Why we multiply by -1

In [33]:
df_h2_flux_per_ch4_flux = -1.0*df_h2_flux.loc['deltaCH4']/df_ch4_flux.loc['deltaCH4']
df_h2_flux_per_ch4_flux.name = 'h2_flux_per_ch4_flux'
df_h2_flux_per_ch4_flux

OSLOCTM3      0.01442
WACCM6-2deg   0.01693
INCA          0.01082
GFDL-emi          nan
GFDL          0.01106
Name: h2_flux_per_ch4_flux, dtype: float64

### HYDROGEN AGWP100 CH4 [mW m-2 yr Tg-1]

Explain why this is AGWP

In [34]:
df_h2_agwp_ch4 = df_surf_ch4_per_h2_flux*spec_rf_ch4
df_h2_agwp_ch4.name = 'h2_agwp_ch4'


#agwp_ch4 = RF per flux H2 (For the per flux table)
df_ch4_rf_per_h2_flux = df_surf_ch4_per_h2_flux*spec_rf_ch4
df_ch4_rf_per_h2_flux.name = 'ch4_rf_per_h2_flux'

### Initialize H2 GWP table

In [35]:
antmod = len(df_h2_agwp_ch4.index)
df_h2_gwp = pd.DataFrame(np.empty([5,antmod])*np.nan,columns=df_h2_agwp_ch4.index,
                         index=['O3','CH4','strat H2O','O3 CH4ind','strat H2O CH4ind'])

### Add methane GWP

In [36]:
df_h2_gwp.loc['CH4'] = df_h2_agwp_ch4/agwp100_CO2

### HYDROGEN AGWP100 strat H2O [mW m-2 yr Tg-1]

Explain that RF/flux equal AGWP..

In [37]:
df_h2_agwp_h2o = df_h2o_rf.loc['10H2']/df_h2_flux.loc['deltaH2']
df_h2_agwp_h2o['GFDL-emi'] = h2o_rf_gfdl['GFDL'].loc['H2+CH4 [mW m-2]']/df_h2_flux['GFDL-emi'].loc['deltaH2']
df_h2_agwp_h2o.name = 'h2_agwp_h2o'

#NBNB GFDL methane induced is included.

In [38]:
#Add to the flux table
df_h2o_rf_per_h2_flux = df_h2o_rf.loc['10H2']/df_h2_flux.loc['deltaH2']
df_h2o_rf_per_h2_flux.name= 'h2o_rf_per_h2_flux'
df_h2o_rf_per_h2_flux['GFDL-emi'] = h2o_rf_gfdl['GFDL'].loc['H2 [mW m-2]']/df_h2_flux['GFDL-emi'].loc['deltaH2']

#Strat H2O RF per methane flux (Move this to the methane part?)
df_h2o_rf_per_ch4_flux = df_h2o_rf.loc['10CH4']/df_ch4_flux.loc['deltaCH4']
df_h2o_rf_per_ch4_flux.name = 'h2o_rf_per_ch4_flux'

### Add stratospheric H2O GWP

In [39]:
df_h2_gwp.loc['strat H2O'] = df_h2_agwp_h2o/agwp100_CO2

### HYDROGEN AGWP100 O3 [mW m-2 yr Tg-1]

In [40]:
df_h2_agwp_o3 = df_ozone_rf.loc['10H2']/df_h2_flux.loc['deltaH2']
df_h2_agwp_o3.name = 'h2_agwp_o3'

#For GFDL-emi include the methane induced part.
df_h2_agwp_o3['GFDL-emi'] = (df_ozone_rf['GFDL-emi'].loc['10H2']+df_ozone_rf['GFDL-emi'].loc['10CH4'])/df_h2_flux['GFDL-emi'].loc['deltaH2']

In [41]:
#Similar, but use only the H2 Ozone RF for GFDL. To be used in the table:
df_ozone_rf_per_h2_flux = df_ozone_rf.loc['10H2']/df_h2_flux.loc['deltaH2']
df_ozone_rf_per_h2_flux.name= 'ozone_rf_per_h2_flux'

In [42]:
#Ozone RF per methane flux (move to the methane part?)
df_ozone_rf_per_ch4_flux = df_ozone_rf.loc['10CH4']/df_ch4_flux.loc['deltaCH4']
df_ozone_rf_per_ch4_flux.name = 'ozone_rf_per_ch4_flux'

### Add Ozone GWP

In [43]:
df_h2_gwp.loc['O3'] = df_h2_agwp_o3/agwp100_CO2

### For the per flux table

In [44]:
df_trop_du_ozone_per_ch4_flux = df_ozone_du_trop.loc['10CH4']/df_ch4_flux.loc['deltaCH4']
df_trop_du_ozone_per_ch4_flux.name = 'trop_du_ozone_per_ch4_flux'

df_strat_du_ozone_per_ch4_flux = df_ozone_du_strat.loc['10CH4']/df_ch4_flux.loc['deltaCH4']
df_strat_du_ozone_per_ch4_flux.name = 'strat_du_ozone_per_ch4_flux'

df_trop_du_ozone_per_h2_flux = df_ozone_du_trop.loc['10H2']/df_h2_flux.loc['deltaH2']
df_trop_du_ozone_per_h2_flux.name = 'trop_du_ozone_per_h2_flux'

df_strat_du_ozone_per_h2_flux = df_ozone_du_strat.loc['10H2']/df_h2_flux.loc['deltaH2']
df_strat_du_ozone_per_h2_flux.name = 'strat_du_ozone_per_h2_flux'


In [45]:
df_strat_du_6060_ozone_per_ch4_flux = df_ozone_du_strat_6060.loc['10CH4']/df_ch4_flux.loc['deltaCH4']
df_strat_du_6060_ozone_per_ch4_flux.name = 'strat_du_6060_ozone_per_ch4_flux'

df_strat_du_6060_ozone_per_h2_flux = df_ozone_du_strat_6060.loc['10H2']/df_h2_flux.loc['deltaH2']
df_strat_du_6060_ozone_per_h2_flux.name = 'strat_du_6060_ozone_per_h2_flux'

df_strat_du_gt60_ozone_per_ch4_flux = df_ozone_du_strat_gt60.loc['10CH4']/df_ch4_flux.loc['deltaCH4']
df_strat_du_gt60_ozone_per_ch4_flux.name = 'strat_du_gt60_ozone_per_ch4_flux'

df_strat_du_gt60_ozone_per_h2_flux = df_ozone_du_strat_gt60.loc['10H2']/df_h2_flux.loc['deltaH2']
df_strat_du_gt60_ozone_per_h2_flux.name = 'strat_du_gt60_ozone_per_h2_flux'

## Methane induced GWP:

### HYDROGEN AGWP100 methane induced O3 [mW m-2 yr Tg-1]

It does not matter here if we use surface concentration, burden, tropospheric concentration?

In [46]:
#Wm-2/ppbCH4*ppbCH4/TgH2yr-1 -> Wm-2/TgH2yr-1
df_h2_agwp_ch4ind_o3 = df_ozone_rf.loc['10CH4']/df_ch4_surfconc.loc['deltaCH4']*df_surf_ch4_per_h2_flux
df_h2_agwp_ch4ind_o3.name = 'h2_agwp_ch4ind_o3'

### Add methane induced O3 GWP

In [47]:
df_h2_gwp.loc['O3 CH4ind'] = df_h2_agwp_ch4ind_o3/agwp100_CO2

### HYDROGEN AGWP100 methane induced strat H2O [mW m-2 yr Tg-1]

In [48]:
df_h2_agwp_ch4ind_h2o = df_h2o_rf.loc['10CH4']/df_ch4_surfconc.loc['deltaCH4']*df_surf_ch4_per_h2_flux
df_h2_agwp_ch4ind_h2o.name = 'h2_agwp_ch4ind_h2o'

### Add methane induced strat H2O GWP

In [49]:
df_h2_gwp.loc['strat H2O CH4ind'] = df_h2_agwp_ch4ind_h2o/agwp100_CO2

# Methane GWP

Initialize CH4 GWP

In [50]:
antmod = len(df_h2_agwp_ch4.index)
df_ch4_gwp = pd.DataFrame(np.empty([4,antmod])*np.nan,columns=df_h2_agwp_ch4.index,
                         index=['O3','CH4','strat H2O','H2'])
#df_ch4_gwp

### Methane AGWP100 O3 [mW m-2 yr Tg-1]

Integrated O3 from CH4 RF [W m-2 yr Tg-1]

In [51]:
df_ch4_agwp_o3 = df_ozone_rf.loc['10CH4']/df_ch4_surfconc.loc['deltaCH4']*df_surf_ch4_per_ch4_flux
df_ch4_agwp_o3.name = 'ch4_agwp_o3'

### Add ozone GWP

In [52]:
df_ch4_gwp.loc['O3'] =df_ch4_agwp_o3/agwp100_CO2 

### Methane AGWP100 Methane [mW m-2 yr Tg-1]

In [53]:
df_ch4_agwp =df_surf_ch4_per_ch4_flux*spec_rf_ch4
df_ch4_agwp.name = 'ch4_agwp'

### Add methane GWP

In [54]:
#Add Methane GWP:
df_ch4_gwp.loc['CH4'] =df_ch4_agwp/agwp100_CO2 

### Methane AGWP100 strat H2O [mW m-2 yr Tg-1]

In [55]:
print(df_h2o_rf.loc['10CH4'])
df_ch4_agwp_h2o = df_h2o_rf.loc['10CH4']/df_ch4_surfconc.loc['deltaCH4']*df_surf_ch4_per_ch4_flux
df_ch4_agwp_h2o.name = 'ch4_agwp_h2o'

OSLOCTM3       9.18
WACCM6-2deg   2.805
INCA            3.4
GFDL            nan
Name: 10CH4, dtype: float64


### Add Strat H2O GWP:

In [56]:
df_ch4_gwp.loc['strat H2O'] = df_ch4_agwp_h2o/agwp100_CO2

### Add GWP via H2

In [57]:
df_ch4_gwp.loc['H2'] = df_h2_flux_per_ch4_flux*df_h2_gwp.sum()

# Main results and tabels

## H2 GWP 100

In [58]:
#Save to file:
#print(outputpath + 'table_h2_gwp.csv')
df_h2_gwp.to_csv(outputpath + 'table_h2_gwp.csv')
df_h2_gwp.loc['total']=df_h2_gwp.sum()
df_h2_gwp

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,GFDL
O3,2.39,1.552,1.693,3.977,4.919
CH4,5.242,5.607,5.652,4.848,4.502
strat H2O,1.727,0.57,0.6055,2.951,nan
O3 CH4ind,2.674,2.919,1.589,nan,1.894
strat H2O CH4ind,0.5992,0.1963,0.2393,nan,nan
total,12.63,10.84,9.778,11.78,11.32


## CH4 GWP 100

In [59]:
df_ch4_gwp.to_csv(outputpath + 'table_ch4_gwp.csv')
df_ch4_gwp.loc['total']=df_ch4_gwp.sum()
df_ch4_gwp

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,GFDL
O3,8.545,8.573,5.431,nan,7.686
CH4,16.75,16.47,19.32,nan,18.27
strat H2O,1.915,0.5766,0.8179,nan,nan
H2,0.1821,0.1836,0.1058,nan,0.1251
total,27.39,25.8,25.68,0,26.08


## Table per flux H2

In [60]:
df_per_flux_h2 = pd.concat([df_h2_flux.loc['deltaH2'],df_surf_h2_per_h2_flux,
                            df_surf_ch4_per_h2_flux,df_ch4_flux_per_h2_flux,df_ch4_rf_per_h2_flux,
                            df_trop_du_ozone_per_h2_flux,df_strat_du_ozone_per_h2_flux,
                            df_strat_du_6060_ozone_per_h2_flux,df_strat_du_gt60_ozone_per_h2_flux,
                            df_ozone_rf_per_h2_flux,df_h2o_rf_per_h2_flux],axis=1, sort=False)

#Save to file:
df_per_flux_h2.to_csv(outputpath + 'table_per_flux_h2.csv')

#Rename the columns:
columns_names={'deltaH2':'Flux H2 [Tg/yr]',
               'surf_h2_per_h2_flux': 'Surf. conc. H2 per flux [ppb yr/Tg]',
               'surf_ch4_per_h2_flux':'Surf. conc. CH4 per flux [ppb yr/Tg]',
               'ch4_flux_per_h2_flux':'Flux CH4/Flux H2 [Tg CH4/Tg H2]',
               'ch4_rf_per_h2_flux':'CH4 RF per flux [mW m-2 yr/ Tg]',
               'trop_du_ozone_per_h2_flux':'Trop. ozone per flux [DU yr/Tg]',
               'strat_du_ozone_per_h2_flux':'Strat. ozone per flux [DU yr/Tg]',
               'strat_du_6060_ozone_per_h2_flux':'Strat. ozone 60S-60N per flux [DU yr/Tg]',
               'strat_du_gt60_ozone_per_h2_flux':'Strat. ozone >60S/N per flux [DU yr/Tg]',
               'ozone_rf_per_h2_flux':'ozone RF per flux [mW m-2 yr/ Tg]',
               'h2o_rf_per_h2_flux':'Strat. H2O RF per flux [mW m-2 yr/ Tg]'}
#Rename column names:
df_per_flux_h2.rename(columns=dict(columns_names),inplace=True) #[df_per_flux_h2.columns])
df_per_flux_h2

,Flux H2 [Tg/yr],Surf. conc. H2 per flux [ppb yr/Tg],Surf. conc. CH4 per flux [ppb yr/Tg],Flux CH4/Flux H2 [Tg CH4/Tg H2],CH4 RF per flux [mW m-2 yr/ Tg],Trop. ozone per flux [DU yr/Tg],Strat. ozone per flux [DU yr/Tg],Strat. ozone 60S-60N per flux [DU yr/Tg],Strat. ozone >60S/N per flux [DU yr/Tg],ozone RF per flux [mW m-2 yr/ Tg],Strat. H2O RF per flux [mW m-2 yr/ Tg]
OSLOCTM3,8.536,6.233,1.085,0.313,0.4807,0.004506,0.001156,0.000855,0.0003005,0.2192,0.1583
WACCM6-2deg,9.921,5.36,1.161,0.3405,0.5142,0.00278,-0.001285,-0.001106,-0.0001793,0.1423,0.05227
INCA,7.195,7.395,1.17,0.2925,0.5183,0.00407,-0.003812,-0.002936,-0.0008762,0.1552,0.05553
GFDL-emi,196.3,6.859,1.004,0.2301,0.4446,0.003714,-0.001153,-0.001113,-4.059e-05,0.1732,0.144
GFDL,7.661,6.847,0.9319,0.2464,0.4128,0.00751,0.008705,0.00318,0.005526,0.4511,nan


## Table per flux CH4

To do: fix the table headings in the table (as for H2 above)

In [61]:
df_per_flux_ch4 = pd.concat([df_ch4_flux.loc['deltaCH4'],df_surf_ch4_per_ch4_flux,
                            df_h2_flux_per_ch4_flux,df_trop_du_ozone_per_ch4_flux,df_strat_du_ozone_per_ch4_flux,
                            df_strat_du_6060_ozone_per_ch4_flux,df_strat_du_gt60_ozone_per_ch4_flux,
                            df_ozone_rf_per_ch4_flux,df_h2o_rf_per_ch4_flux],axis=1,sort=False)



               
#Save to file:
df_per_flux_ch4.to_csv(outputpath + 'table_per_flux_ch4.csv')

#Rename the columns:
columns_names={'deltaCH4':'Flux CH4 [Tg/yr]',
               'surf_ch4_per_ch4_flux':'Surf. conc. CH4 per flux [ppb yr/Tg]',
               'h2_flux_per_ch4_flux':'Flux H2/Flux CH4 [Tg H2/Tg CH4]',
               'trop_du_ozone_per_ch4_flux':'Trop. ozone per flux [DU yr/Tg]',
               'strat_du_ozone_per_ch4_flux':'Strat. ozone per flux [DU yr/Tg]',
               'strat_du_6060_ozone_per_ch4_flux':'Strat. ozone 60S-60N per flux [DU yr/Tg]',
               'strat_du_gt60_ozone_per_ch4_flux':'Strat. ozone >60S/N per flux [DU yr/Tg]',
               'ozone_rf_per_ch4_flux':'ozone RF per flux [mW m-2 yr/ Tg]',
               'h2o_rf_per_ch4_flux':'Strat H2O RF per flux [mW m-2 yr/ Tg]'}
               
#Rename column names:
df_per_flux_ch4.rename(columns=dict(columns_names),inplace=True) #[df_per_flux_h2.columns])
df_per_flux_ch4          

,Flux CH4 [Tg/yr],Surf. conc. CH4 per flux [ppb yr/Tg],Flux H2/Flux CH4 [Tg H2/Tg CH4],Trop. ozone per flux [DU yr/Tg],Strat. ozone per flux [DU yr/Tg],Strat. ozone 60S-60N per flux [DU yr/Tg],Strat. ozone >60S/N per flux [DU yr/Tg],ozone RF per flux [mW m-2 yr/ Tg],Strat H2O RF per flux [mW m-2 yr/ Tg]
OSLOCTM3,52.29,3.467,0.01442,0.01589,0.01586,0.01205,0.004446,0.7835,0.1756
WACCM6-2deg,53.05,3.409,0.01693,0.0162,0.01457,0.0115,0.003073,0.7861,0.05288
INCA,45.33,3.999,0.01082,0.0119,0.007774,0.006103,0.001671,0.498,0.075
GFDL-emi,nan,nan,nan,nan,nan,nan,nan,nan,nan
GFDL,47.3,3.782,0.01106,0.01399,0.01642,0.01206,0.004367,0.7048,nan


## H2 budget tabel

In [62]:
df_budget_h2 = pd.concat([df_h2_burden.loc['CTRL'],df_h2_lifetime.loc['CTRL'],
                          df_h2_atmloss.loc['CTRL'],df_h2_drydep.loc['CTRL'],
                          df_h2_atmprod.loc['CTRL'],df_h2_atm_lifetime.loc['CTRL'],
                          df_h2_estemis],axis=1)

df_budget_h2.columns = ['H2 burden [Tg]',
                        'H2 lifetime [yrs]',
                        'H2 atm.loss[Tg/yr]',
                        'H2 drydep [Tg/yr]',
                        'H2 atm. prod [Tg/yr]',
                        'H2 atmospheric lifetime [yrs]',
                        'H2 estimated emissions [Tg/yr]']

df_budget_h2.to_csv(outputpath + 'table_budget_h2.csv')
df_budget_h2

,H2 burden [Tg],H2 lifetime [yrs],H2 atm.loss[Tg/yr],H2 drydep [Tg/yr],H2 atm. prod [Tg/yr],H2 atmospheric lifetime [yrs],H2 estimated emissions [Tg/yr]
OSLOCTM3,196.2,2.232,28.39,59.52,56.32,6.911,31.58
WACCM6-2deg,194.4,1.906,29.51,72.46,34.63,6.587,67.34
INCA,194.4,2.62,22.13,52.06,46.68,8.786,27.51
GFDL-emi,191.7,2.497,22.12,54.66,44.68,8.665,32.1
GFDL,187.2,2.391,21.53,56.77,44.66,8.693,33.64


## CH4 budget tabel

In [63]:
df_budget_ch4 = pd.concat([df_ch4_burden.loc['CTRL'],df_ch4_lifetime.loc['CTRL'],
                           df_ch4_loss.loc['CTRL']],axis=1)
df_budget_ch4.columns = ['CH4 burden [Tg]','CH4 lifetime [yrs]','CH4 atm.loss[Tg/yr]'] 
df_budget_ch4

,CH4 burden [Tg],CH4 lifetime [yrs],CH4 atm.loss[Tg/yr]
OSLOCTM3,"4,995",7.314,683
WACCM6-2deg,"5,006",6.744,742.3
INCA,"4,994",9.088,549.6
GFDL-emi,"4,935",8.476,582.3
GFDL,"4,857",8.52,570


# Other results/checks:

## Methane feedback factor:

### Atmospheric mass conversion CH4  [Tg/ppb] (from perturbations)

In [64]:
df_ch4_burden_per_conc  = df_ch4_burden.loc['deltaCH4']/df_ch4_surfconc.loc['deltaCH4']
df_ch4_burden_per_conc.name = 'ch4_burden_per_conc'
df_ch4_burden_per_conc

OSLOCTM3      2.758
WACCM6-2deg   2.775
INCA          2.755
GFDL-emi        nan
GFDL          2.717
Name: ch4_burden_per_conc, dtype: float64

### Increase per unit flux w/o feedback = integrated decay [ppb yr/Tg]

Add explanation... 

In [65]:
df_w_o_feedback =df_ch4_lifetime.loc['CTRL']/df_ch4_burden_per_conc #Lifetime [yr] / [Tg/ppb] 
df_w_o_feedback

OSLOCTM3      2.651
WACCM6-2deg    2.43
INCA          3.299
GFDL-emi        nan
GFDL          3.135
dtype: float64

### Feedback factor: increase CH4 with feedback/ increase CH4 without feedback

In [66]:
df_feedback_factor_ch4 = df_surf_ch4_per_ch4_flux/df_w_o_feedback
df_feedback_factor_ch4.name = 'feedback_factor_ch4'
df_feedback_factor_ch4

OSLOCTM3      1.308
WACCM6-2deg   1.403
INCA          1.212
GFDL-emi        nan
GFDL          1.206
Name: feedback_factor_ch4, dtype: float64

## Hydrogen feedback factor:

### Atmospheric mass conversion H2  [Tg/ppb] (from perturbations)

In [67]:
df_h2_burden_per_conc  = df_h2_burden.loc['deltaH2']/df_h2_surfconc.loc['deltaH2']
df_h2_burden_per_conc.name = 'h2_burden_per_conc'
df_h2_burden_per_conc

OSLOCTM3      0.3506
WACCM6-2deg   0.3498
INCA          0.3496
GFDL-emi      0.3445
GFDL          0.3464
Name: h2_burden_per_conc, dtype: float64

### Increase per unit flux w/o feedback = integrated decay [ppb yr/Tg]

In [68]:
df_w_o_feedback_h2 =df_h2_lifetime.loc['CTRL']/df_h2_burden_per_conc #Lifetime [yr] / [Tg/ppb] 
df_w_o_feedback_h2

OSLOCTM3      6.367
WACCM6-2deg    5.45
INCA          7.495
GFDL-emi      7.247
GFDL          6.903
dtype: float64

### Feedback factor: increase H2 with feedback/ increase H2 without feedback

In [69]:
df_feedback_factor_h2 = df_surf_h2_per_h2_flux/df_w_o_feedback_h2
df_feedback_factor_h2.name = 'feedback_factor_h2'
df_feedback_factor_h2
#Fabien wrote in the paper about feedback factor less than 1.

OSLOCTM3      0.9789
WACCM6-2deg   0.9835
INCA          0.9866
GFDL-emi      0.9465
GFDL          0.9919
Name: feedback_factor_h2, dtype: float64